In [1]:
import math
import sympy # to check primality

from Rings import Ztau, Zomega

An implementation of the Tonelli-Shanks algorithm as found here
https://rosettacode.org/wiki/Tonelli-Shanks_algorithm#Python

In [29]:
def legendre(a, p):
    return pow(a, (p - 1) // 2, p)

def modular_sqrt(n, p):
    assert legendre(n, p) == 1, "not a square (mod p)"
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1
    if s == 1:
        return pow(n, (p + 1) // 4, p)
    for z in range(2, p):
        if p - 1 == legendre(z, p):
            break
    c = pow(z, q, p)
    r = pow(n, (q + 1) // 2, p)
    t = pow(n, q, p)
    m = s
    t2 = 0
    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 - 1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r

In [30]:
def splitting_root(xi):
    # Find a square root of tau-2 modulo xi
    p = xi.N()
    b1 = pow(xi.b, -1, p)
    print(b1)
    print(-xi.a*b1 - 2)
    return modular_sqrt(-xi.a*b1 - 2, p)

Test splitting_root

In [32]:
xi = Ztau(124233240587, -201011676794)
print(splitting_root(xi))

412376737179780857
-51230898402537783079618043061
373413958110469043


In [5]:
def unit_dlog(u):
    # Takes a unit u in Ztau and returns (s,k) where u = s*tau^k
    s, k, a, b = 1, 0, u.a, u.b

    if a < 0:
        a, b, s = -a, -b, -s

    mu = a*b

    # repeatedly multiply by tau or tau^-1 until the remainder has |mu| <= 1
    while abs(mu) > 1:
        if mu > 1:
            a, b, k = b, a-b, k-1
        else:
            a, b, k = a+b, a, k+1
        mu = a*b

    if a == -1 and b == 0:
        s = -s
    elif a == 0 and b == 1:
        k = k+1
    elif a == 0 and b == -1:
        s, k = -s, k+1
    elif a == 1 and b == -1:
        k = k+2
    elif a == -1 and b == 1:
        s, k = -s, k+2
    elif a == 1 and b == 1:
        k = k-1
    elif a == -1 and b == -1:
        s, k = -s, k-1

    return (s, k)

Test dlog

In [6]:
u = Ztau(0, -1)
print(u**5)
print(unit_dlog(u**5))

(3 + -5τ)
(-1, 5)


In [7]:
def easy_factor(xi):
    # factors out the integer part as well as (2 - tau) if it exists
    c = math.gcd(xi.a, xi.b)
    a1, b1 = int(xi.a/c), int(xi.b/c)
    xi_1 = Ztau(a1, b1)

    factors = []

    if math.sqrt(c).is_integer():
        if c > 1:
            factors = [(Ztau(int(math.sqrt(c)),0),2)]
    else:
        if math.sqrt(c/5).is_integer():
            factors = [(Ztau(int(math.sqrt(c/5)), 0),2), (Ztau(5,0),1)]
        else:
            factors = [(xi, 1)]
    
    n = xi_1.N()
    if n%5 == 0:
        p1 = Ztau(2, -1)
        xi_2 = xi_1//p1
        factors.append((p1, 1))
        factors.append((xi_2, 1))
    else:
        factors.append((xi_1, 1))
    
    return factors


In [8]:
def easy_solveable(factors):
    for i in range(len(factors)):
        (xi, k) = factors[i]
        if k%2 == 1:
            if xi != Ztau(5,0):
                p = xi.N()
                r = p%5
                if (not sympy.isprime(p)) or (r != 0 and r != 1):
                    return False
    
    return True


In [9]:
xi = Ztau(1222 , -20725)
print(xi.N())
print(xi.value())

-402706391
-11586.754416841572


Test easy_factor and easy_solveable

In [10]:
xi = Ztau(760, -780)
factors = easy_factor(xi)
for i in range(len(factors)):
    print(factors[i][0], factors[i][1])
print(easy_solveable(factors))

(2 + 0τ) 2
(5 + 0τ) 1
(2 + -1τ) 1
(15 + -8τ) 1
True


In [11]:
bin = Zomega(1,1,0,0)

def binary_gcd(a,b):
    print(a, b)
    print(a.N(), b.N())
    if a.N() == 0:
        return b
    if b.N() == 0:
        return a
    
    # the integer remainder modulo 1+w is given by substituting w = -1
    a_rem = (a.a-a.b+a.c-a.d)%5
    b_rem = (b.a-b.b+b.c-b.d)%5
    if a_rem == 0 and b_rem == 0:
        a1 = a//bin
        b1 = b//bin
        return bin * binary_gcd(a1, b1)
    
    if a_rem == 0:
        a1 = a//bin
        return binary_gcd(a1, b)
    
    if b_rem == 0:
        b1 = b//bin
        return binary_gcd(a, b1)
    
    u, v = Zomega(1,0,0,0), Zomega(1,0,0,0)
    if a_rem != 0 and b_rem != 0:
        if a_rem == 2:
            u = Zomega(0,1,0,1)
        elif a_rem == 3:
            u = Zomega(0,-1,0,-1)
        elif a_rem == 4:
            u = Zomega(-1,0,0,0)
        
        if b_rem == 2:
            v = Zomega(0,1,0,1)
        elif b_rem == 3:
            v = Zomega(0,-1,0,-1)
        elif b_rem == 4:
            v = Zomega(-1,0,0,0)

    if a.N() <= b.N():
        c = a
    else:
        c = b

    return binary_gcd(c, u*a - v*b)

In [12]:
def gcd(a,b):
    if a.N() == 0:
        return b
    if b.N() == 0:
        return a
    
    if a.N() <= b.N():
        q = b//a
        r = b - q*a
        print(b, a, q, r, r.N())
        return gcd(a,r)
    else:
        q = a//b
        r = a - q*b
        print(a, b, q, r, r.N())
        return gcd(b,r)

In [13]:
xi = Zomega(13, 0, -2, 2)
y = Zomega(11, -2, 1, -1)

g = gcd(xi,y)
print(g)

(13 + 0ω + -2ω² + 2ω³) (11 + -2ω + 1ω² + -1ω³) (1 + 0ω + 0ω² + 0ω³) (2 + 2ω + -3ω² + 3ω³) 191
(11 + -2ω + 1ω² + -1ω³) (2 + 2ω + -3ω² + 3ω³) (4 + -2ω + 1ω² + -3ω³) (0 + 0ω + 0ω² + 0ω³) 0
(2 + 2ω + -3ω² + 3ω³)


In [14]:
print(Zomega(15,0,-8,8)*Zomega(5,0,2,-2) + Zomega(5,-2,-5,5))
print(Zomega(5,-2,-5,5)*Zomega(0,4,1,5))

(64 + -2ω + 1ω² + -1ω³)
(15 + 0ω + -8ω² + 8ω³)


In [15]:
def solve_norm_equation(xi):
    # Outputs x from Zomega which solves |x|^2 = xi

    # first check solveability
    if xi.value() < 0 or xi.dot().value() < 0:
        return Zomega(0,0,0,0)
    factors = easy_factor(xi)
    if not easy_solveable(factors):
        return Zomega(0,0,0,0)
    
    print("Factors --------------------")
    for i in range(len(factors)):
        print(factors[i][0], factors[i][1])
    
    print("--------------------------")

    x = Zomega(1,0,0,0)

    # solve for each prime factor separately, then multiply solutions together
    for i in range(len(factors)):
        (xi_i, m) = factors[i]
        x = x * (Zomega.fromTau(xi_i)**(int(m/2)))
        if m%2 == 1:
            if xi_i == Ztau(5,0):
                x = x * (Zomega.fromTau(Ztau(1, 2)))
            elif xi_i == Ztau(2, -1):
                x = x*Zomega(-1,2,-1,1)
            else:
                print("xi_i: " + str(xi_i))
                M = splitting_root(xi_i)
                print("M: " + str(M))
                print(Zomega.fromTau(xi_i), Zomega(M,0,0,0) - Zomega(-1,2,-1,1))
                y = gcd(Zomega.fromTau(xi_i), Zomega(M,0,0,0) - Zomega(-1,2,-1,1))
                print("gcd: " + str(y))
                u = xi_i // y.Ni()
                print(u)
                (s,m) = unit_dlog(u)
                print(s,m)
                x = x * (Zomega.fromTau(Ztau(0,1)**int(m/2))) * y
        
        print(x)
    
    return x       



In [34]:
print(Ztau(102,-1)//Ztau(13,-2))
xi = Zomega(13, 0, -2, 2)
y = Zomega(11, -2, 1, -1)
print(xi//y)
print(y.Ni())
xi = Zomega(-384, 809, -752,224)
print(xi.Ni())

(8 + 1τ)
(1 + 0ω + 0ω² + 0ω³)
(102 + -1τ)
(330145 + -531472τ)


In [33]:
xi = Ztau(124233240587, -201011676794)
x = solve_norm_equation(xi)
print(x)
print(x.Ni())


Factors --------------------
(124233240587 + -201011676794τ) 1
--------------------------
xi_i: (124233240587 + -201011676794τ)
412376737179780857
-51230898402537783079618043061
M: 373413958110469043
(124233240587 + 0ω + -201011676794ω² + 201011676794ω³) (373413958110469044 + -2ω + 1ω² + -1ω³)
(373413958110469044 + -2ω + 1ω² + -1ω³) (124233240587 + 0ω + -201011676794ω² + 201011676794ω³) (226645535011 + 0ω + 140074130588ω² + -140074130588ω³) (-77179047541 + -2ω + 124878284707ω² + -124878284707ω³) 41740196074269884741783903621405
(124233240587 + 0ω + -201011676794ω² + 201011676794ω³) (-77179047541 + -2ω + 124878284707ω² + -124878284707ω³) (-38 + 0ω + -22ω² + 22ω³) (-61248302461 + -32ω + 99101832572ω² + -99101832616ω³) 125325741119258965137376304021
(-77179047541 + -2ω + 124878284707ω² + -124878284707ω³) (-61248302461 + -32ω + 99101832572ω² + -99101832616ω³) (11 + 0ω + 6ω² + -6ω³) (1941284026 + 422ω + -3141063195ω² + 3141063871ω³) 44969872467274129822597051
(-61248302461 + -32ω + 99101832